<a href="https://www.kaggle.com/code/stutiiikatariaaa/lightgbm?scriptVersionId=270015423" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv('/kaggle/input/solargeneration/dataset.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d-%m-%Y %H:%M')

In [ ]:
# Feature engineering
df['hour'] = df['date_time'].dt.hour
df['day_of_week'] = df['date_time'].dt.dayofweek

# Select features and target
features = ['temperature', 'Ground_radiation_intensity ', 'hour']
X = df[features]
y = df['Photovoltaic_power_generation']

In [ ]:
# Split data (time-series split)
train_size = int(len(X) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold

# Initialize scaler
scaler = StandardScaler()

# Fit on training data only (avoid data leakage)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use same scaler

# Convert back to DataFrame for readability
X_train = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

LightGBM without Hyperparameter tuning

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMRegressor

# Initialize LightGBM Regressor with default/fixed parameters
model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=-1,
    num_leaves=31,
    min_child_samples=20,
    subsample=1.0,
    colsample_bytree=1.0,
    random_state=42,
    verbose=-1
)

# Fit the model
model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nDefault LightGBM Test Set Evaluation:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")



In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(18, 6))
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}, index=y_test.index).head(500).plot(ax=plt.gca())
plt.title("Default LightGBM: Actual vs Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index", fontsize=12)
plt.ylabel("Power Generation (kW)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


LightGBM with Hyperparameter tuning

In [ ]:
# Define hyperparameter search space
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.005, 0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
    'num_leaves': [15, 31, 50, 70],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

In [ ]:
# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=40,
    scoring='r2',
    cv=3,
    verbose=0,
    random_state=42,
    n_jobs=-1
)

In [ ]:
# Fit the model
random_search.fit(X_train_scaled, y_train)
best_model = random_search.best_estimator_

# Print best parameters
print("Best Hyperparameters:")
print(random_search.best_params_)

# Predict and evaluate
y_pred = best_model.predict(X_test_scaled)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nTest Set Evaluation:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(18, 6))
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}, index=y_test.index).head(500).plot(ax=plt.gca())
plt.title("LightGBM with RandomizedSearchCV: Actual vs Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index", fontsize=12)
plt.ylabel("Power Generation (kW)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()